# Collect All Functions
------------------
This project comes with functions that collect all STIX objects from all ATT&CK Matrices at once. These functions help collect more with less API call requests.

## Import Extra Libraries

In [1]:
import json
import pandas

## Initialize ATT&CK Client & Download STIX Data

In [ ]:
from attackcti import MitreAttackClient

# Downloads latest STIX 2.1 (enterprise+mobile+ics) to .attackcti/stix-2.1 (reuses files if present)
lift = MitreAttackClient.from_attack_stix_data()

## Get All Techniques
We can extract all STIX objects of type `attack-pattern` (technique) across all ATT&CK matrices.
* By default, this function removes `deprecated` and `revoked` techniques. If you want to keep those techniques in the results, you can run the function with the parameter `skip_revoked_deprecated=False`.
* By default, this function also includes all techniques and sub-techniques. If you want to only get techniques that are not sub-techniques, you can run the function with the parameter `include_subtechniques=False`.

In [3]:
techniques = lift.get_techniques()

In [4]:
print("Number of Techniques in ATT&CK")
len(techniques)

Number of Techniques in ATT&CK


898

By default, the data returned by the available functions in the attackcti library is of type **stix2**. However, if you want to interact with libraries such as **Pandas**, it needs to be of type **dict**

In [6]:
all_techniques = []
for t in techniques:
    all_techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(all_techniques)
df.reindex(['created','name', 'x_mitre_platforms'], axis=1)[0:5]


,created,name,x_mitre_platforms
0,2020-01-14T17:18:32.126Z,Extra Window Memory Injection,[Windows]
1,2019-11-27T14:58:00.429Z,Scheduled Task,[Windows]
2,2022-09-30T21:18:41.930Z,Socket Filters,"[Linux, macOS, Windows]"
3,2020-02-20T21:01:25.428Z,Archive via Utility,"[Linux, macOS, Windows]"
4,2020-02-11T18:28:44.950Z,VNC,"[Linux, Windows, macOS]"


We can now access the schema of the dataframe

In [7]:
list(df)

['type',
 'spec_version',
 'id',
 'created_by_ref',
 'created',
 'modified',
 'name',
 'description',
 'kill_chain_phases',
 'external_references',
 'object_marking_refs',
 'x_mitre_attack_spec_version',
 'x_mitre_deprecated',
 'x_mitre_detection',
 'x_mitre_domains',
 'x_mitre_is_subtechnique',
 'x_mitre_modified_by_ref',
 'x_mitre_platforms',
 'x_mitre_version',
 'x_mitre_contributors',
 'x_mitre_remote_support',
 'x_mitre_impact_type',
 'x_mitre_tactic_type']

Showing one technique example:

In [8]:
json.loads(techniques[0].serialize())

{'type': 'attack-pattern',
 'spec_version': '2.1',
 'id': 'attack-pattern--0042a9f5-f053-4769-b3ef-9ad018dfa298',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2020-01-14T17:18:32.126Z',
 'modified': '2025-10-24T17:48:19.059Z',
 'name': 'Extra Window Memory Injection',
 'description': "Adversaries may inject malicious code into process via Extra Window Memory (EWM) in order to evade process-based defenses as well as possibly elevate privileges. EWM injection is a method of executing arbitrary code in the address space of a separate live process. \n\nBefore creating a window, graphical Windows-based processes must prescribe to or register a windows class, which stipulate appearance and behavior (via windows procedures, which are functions that handle input/output of data).(Citation: Microsoft Window Classes) Registration of new windows classes can include a request for up to 40 bytes of EWM to be appended to the allocated memory of each instance of th

## Enrich Techniques with Detection Context (Strategies → Analytics → Log Source References)

ATT&CK now models detection strategies and analytics as discrete objects. Instead of relying on deprecated technique fields (for example x_mitre_data_sources), ATTACKCTI can enrich each technique by traversing the detection model:

```
x-mitre-detection-strategy --detects--> attack-pattern
x-mitre-detection-strategy.x_mitre_analytic_refs -> x-mitre-analytic
x-mitre-analytic.x_mitre_log_source_references -> log source reference dicts (with data component refs)
```

Use `get_techniques(enrich_detections=True)` to attach `x_attackcti_detection_strategies` to each technique. Each strategy contains `x_attackcti_analytics`, and each analytic contains `x_attackcti_log_sources` with `x_mitre_data_component_ref` references.

In [9]:
techniques = lift.get_techniques(enrich_detections=True)

In [10]:
json.loads(techniques[0].serialize())

{'type': 'attack-pattern',
 'spec_version': '2.1',
 'id': 'attack-pattern--0042a9f5-f053-4769-b3ef-9ad018dfa298',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2020-01-14T17:18:32.126Z',
 'modified': '2025-12-19T03:55:02.966463Z',
 'name': 'Extra Window Memory Injection',
 'description': "Adversaries may inject malicious code into process via Extra Window Memory (EWM) in order to evade process-based defenses as well as possibly elevate privileges. EWM injection is a method of executing arbitrary code in the address space of a separate live process. \n\nBefore creating a window, graphical Windows-based processes must prescribe to or register a windows class, which stipulate appearance and behavior (via windows procedures, which are functions that handle input/output of data).(Citation: Microsoft Window Classes) Registration of new windows classes can include a request for up to 40 bytes of EWM to be appended to the allocated memory of each instance of

This function returns a list of techniques with the `x_mitre_data_sources` attribute as a list of STIX objects representing data sources with their respective data components depending on the technique's detection context.

In [11]:
techniques = lift.get_techniques(enrich_detections=True)

technique = techniques[0]
details = technique.get("x_attackcti_detection_strategies", [])

for strategy in details:
    print(f"Detection strategy: {strategy.get('name')} ({strategy.get('id')})")
    for analytic in strategy.get("x_attackcti_analytics", []):
        print(f"  Analytic: {analytic.get('name')} ({analytic.get('id')})")
        for log_source in analytic.get("x_attackcti_log_sources", []):
            comp_ref = log_source.get("x_mitre_data_component_ref")
            print(f"    Log source: {log_source.get('name')} - {log_source.get('channel')} (component ref {comp_ref})")

Detection strategy: Detection Strategy for Extra Window Memory (EWM) Injection on Windows (x-mitre-detection-strategy--1a8d87f1-48ca-4929-a5cc-2b2a03983f12)
  Analytic: Analytic 0608 (x-mitre-analytic--6ec034ac-289d-48d1-b310-021dfbf7087b)
    Log source: WinEventLog:Sysmon - EventCode=10 (component ref x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
    Log source: etw:Microsoft-Windows-Win32k - SetWindowLong, SetClassLong, NtUserMessageCall, SendNotifyMessage, PostMessage (component ref x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
    Log source: WinEventLog:Security - EventCode=4688 (component ref x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379077)


## Enrich Techniques with Data Components (Resolve References to Full Objects)

Log source references include `x_mitre_data_component_ref`, which points to a full `x-mitre-data-component` object. If you want those full objects embedded into the same hierarchy, call:

`get_techniques(enrich_detections=True, enrich_data_components=True)`

This adds `x_attackcti_data_component` under each `x_attackcti_log_sources` entry when the component can be resolved.

In [12]:
techniques = lift.get_techniques(enrich_detections=True, enrich_data_components=True)

In [13]:
technique = techniques[0]
details = technique.get("x_attackcti_detection_strategies", [])

for strategy in details:
    print(f"Detection strategy: {strategy.get('name')} ({strategy.get('id')})")
    for analytic in strategy.get("x_attackcti_analytics", []):
        print(f"  Analytic: {analytic.get('name')} ({analytic.get('id')})")
        for log_source in analytic.get("x_attackcti_log_sources", []):
            comp_ref = log_source.get("x_mitre_data_component_ref")
            print(f"    Log source: {log_source.get('name')} - {log_source.get('channel')} (component ref {comp_ref})")

            comp = log_source.get("x_attackcti_data_component")
            if isinstance(comp, dict):
                print(f"      Data component: {comp.get('name')} ({comp.get('id')})")


Detection strategy: Detection Strategy for Extra Window Memory (EWM) Injection on Windows (x-mitre-detection-strategy--1a8d87f1-48ca-4929-a5cc-2b2a03983f12)
  Analytic: Analytic 0608 (x-mitre-analytic--6ec034ac-289d-48d1-b310-021dfbf7087b)
    Log source: WinEventLog:Sysmon - EventCode=10 (component ref x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
      Data component: Process Access (x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
    Log source: etw:Microsoft-Windows-Win32k - SetWindowLong, SetClassLong, NtUserMessageCall, SendNotifyMessage, PostMessage (component ref x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
      Data component: OS API Execution (x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
    Log source: WinEventLog:Security - EventCode=4688 (component ref x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379077)
      Data component: Process Creation (x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379

## Get All Groups
We can also extract all the available groups across all ATT&CK matrices at once.

In [14]:
groups = lift.get_groups()

In [15]:
print("Number of Groups in ATT&CK")
len(groups)

Number of Groups in ATT&CK


176

In [16]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,name,description,aliases,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_contributors,x_mitre_deprecated,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_version
0,intrusion-set,2.1,intrusion-set--01e28736-2ffc-455b-9880-ed4d140...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2021-01-06T17:46:35.134Z,2024-10-28T19:11:56.485Z,Indrik Spider,[Indrik Spider](https://attack.mitre.org/group...,"[Indrik Spider, Evil Corp, Manatee Tempest, DE...","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,"[Jennifer Kim Roman, CrowdStrike, Liran Ravich...",False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,4.1
1,intrusion-set,2.1,intrusion-set--b7f627e2-0817-4cd5-8d50-e75f8aa...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-02-23T15:31:38.829Z,2025-04-16T20:37:32.806Z,LuminousMoth,[LuminousMoth](https://attack.mitre.org/groups...,[LuminousMoth],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,"[Kyaw Pyiyt Htet, @KyawPyiytHtet, Zaw Min Htun...",False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0
2,intrusion-set,2.1,intrusion-set--918da025-04bd-48af-b6c4-f3e4d1b...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2025-10-15T18:54:55.000Z,2025-10-24T04:01:48.210Z,Medusa Group,[Medusa Group](https://attack.mitre.org/groups...,[Medusa Group],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.3.0,NaN,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0
3,intrusion-set,2.1,intrusion-set--dd2d9ca6-505b-4860-a604-233685b...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2020-05-12T18:15:29.396Z,2025-03-12T20:33:21.597Z,Wizard Spider,[Wizard Spider](https://attack.mitre.org/group...,"[Wizard Spider, UNC1878, TEMP.MixMaster, Grim ...","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,"[Edward Millington, Oleksiy Gayda]",False,"[enterprise-attack, ics-attack]",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,4.0


## Get All Software
We can extract all Enterprise, Mobile and ICS (Software Malware & Tools)

In [17]:
software = lift.get_software()

In [18]:
print("Number of Software in ATT&CK")
len(software)

Number of Software in ATT&CK


910

In [19]:
software_list = []
for t in software:
    software_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(software_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,name,description,is_family,external_references,object_marking_refs,x_mitre_aliases,x_mitre_attack_spec_version,x_mitre_deprecated,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_platforms,x_mitre_version,x_mitre_contributors
0,malware,2.1,malware--007b44b6-e4c5-480b-b5b9-56f2081b1b7b,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:32:40.801Z,2025-04-16T20:37:51.573Z,HDoor,[HDoor](https://attack.mitre.org/software/S006...,True,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[HDoor, Custom HDoor]",3.2.0,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],1.0,NaN
1,malware,2.1,malware--00806466-754d-44ea-ad6f-0caf59cb8556,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2018-10-17T00:14:20.652Z,2024-04-10T22:28:21.746Z,TrickBot,[TrickBot](https://attack.mitre.org/software/S...,True,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[TrickBot, Totbrick, TSPY_TRICKLOAD]",3.2.0,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],2.2,"[Daniyal Naeem, BT Security, Cybereason Noctur..."
2,malware,2.1,malware--00936d7a-451d-4a9c-88fc-05b141aa2d3f,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2025-02-19T20:07:23.223Z,2025-04-15T19:46:33.009Z,cd00r,[cd00r](https://attack.mitre.org/software/S120...,True,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[cd00r],3.2.0,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Network Devices],1.0,NaN
3,malware,2.1,malware--00c3bfcb-99bd-4767-8c03-b08f585f5c8a,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:33:19.746Z,2025-04-25T14:42:30.325Z,PowerDuke,[PowerDuke](https://attack.mitre.org/software/...,True,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[PowerDuke],3.2.0,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],1.2,NaN


## Get All Relationships
We can also get all relationships from all ATT&CK matrices with one API request.

In [20]:
relationships = lift.get_relationships()

In [21]:
print("Number of Relationships in ATT&CK")
len(relationships)

Number of Relationships in ATT&CK


23205

In [22]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,relationship_type,description,source_ref,target_ref,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_deprecated,x_mitre_modified_by_ref,x_mitre_version
0,relationship,2.1,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2021-04-27T01:56:35.810Z,2025-04-28T15:31:30.051Z,uses,[Explosive](https://attack.mitre.org/software...,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,attack-pattern--707399d6-ab3e-4963-9315-d9d381...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,NaN
1,relationship,2.1,relationship--0005fb3b-274a-4ac1-8fb2-51366fcd...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-09-01T21:33:59.394Z,2025-04-16T21:52:51.429Z,mitigates,Consider blocking download/transfer and execut...,course-of-action--21da4fd4-27ad-4e9c-b93d-0b9b...,attack-pattern--43c9bc06-715b-42db-972f-52d25c...,NaN,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,NaN
2,relationship,2.1,relationship--000aa4d0-315e-40d7-b2b6-76e91ecf...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2021-09-15T18:02:37.631Z,2025-04-28T15:31:30.641Z,uses,[Indrik Spider](https://attack.mitre.org/group...,intrusion-set--01e28736-2ffc-455b-9880-ed4d140...,attack-pattern--65f2d882-3f41-4d48-8a06-29af77...,[{'source_name': 'Symantec WastedLocker June 2...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,NaN
3,relationship,2.1,relationship--00192a5f-9dc0-445a-b010-d77bd08a...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2021-05-26T14:50:00.881Z,2025-04-28T15:31:30.812Z,uses,[SombRAT](https://attack.mitre.org/software/S0...,malware--425771c5-48b4-4ecd-9f95-74ed3fc9da59,attack-pattern--bf176076-b789-408e-8cba-7275e8...,[{'source_name': 'BlackBerry CostaRicto Novemb...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,NaN


## Get All Data Sources
Now that `data sources` are part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [23]:
data_sources = lift.get_data_sources()

In [24]:
print("Number of Data Sources in ATT&CK")
len(data_sources)

Number of Data Sources in ATT&CK


0

## Get All Data Components
Now that `data components` are also part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [25]:
data_components = lift.get_data_components()

In [26]:
print("Number of data components in ATT&CK")
len(data_components)

Number of data components in ATT&CK


119

In [27]:
data_components[0]

{'type': 'x-mitre-data-component',
 'spec_version': '2.1',
 'id': 'x-mitre-data-component--02d090b6-8157-48da-98a2-517f7edd49fc',
 'created': '2021-10-20T15:05:19.274Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'revoked': False,
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/datacomponents/DC0084',
   'external_id': 'DC0084'}],
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'modified': '2025-11-12T22:03:39.105Z',
 'name': 'Active Directory Credential Request',
 'description': 'Requests for authentication credentials via Kerberos or other methods like NTLM and LDAP queries. Examples:\n\n- Kerberos TGT and Service Tickets (Event IDs 4768, 4769)\n- NTLM Authentication Events\n- LDAP Bind Requests.',
 'x_mitre_data_source_ref': '',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'x_mitre_deprecated': False,
 'x_mitre_domains': ['enterprise-attack